In [1]:
import pandas as pd
import numpy as np
import glob
import os

In [2]:
data_path = "/Users/jahrios/Documents/Stanford/PoldrackLab/Projects/SharedControl/Data/raw/"
exp_stage = "test"
task = "simple_stop"

pattern = os.path.join(data_path, exp_stage, '*', task, '*.csv')

data_files = glob.glob(pattern)

In [3]:
def compute_SSRT(df, exp_id, max_go_rt = 2000, violation_flag = False):
    
    df = df.query('exp_stage == "test"')
    
    if exp_id == 'stop_signal':
        go_trials = df.loc[df.SS_trial_type == 'go']
        stop_df = df.loc[df.SS_trial_type == 'stop']
    elif exp_id in ['motor_selective_stop_signal', 'stim_selective_stop_signal']:
        if violation_flag and exp_id == 'stim_selective_stop_signal':
            go_trials = df.loc[df.condition == 'ignore']
            stop_df = df.loc[df.condition == 'stop']
        else:
            go_trials = df.loc[df.condition == 'go']
            stop_df = df.loc[df.condition == 'stop']

        
    go_replacement_df = go_trials.where(go_trials['rt'] != -1, max_go_rt)
    sorted_go = go_replacement_df.rt.sort_values(ascending = True, ignore_index=True)
    stop_failure = stop_df.loc[stop_df.rt != -1]

    p_respond = len(stop_failure)/len(stop_df)
    avg_SSD = stop_df.SS_delay.mean()

    nth_index = int(np.rint(p_respond*len(sorted_go))) - 1

    if nth_index < 0:
        nth_RT = sorted_go[0]
    elif nth_index >= len(sorted_go):
        nth_RT = sorted_go[-1]
    else:
        nth_RT = sorted_go[nth_index]

    SSRT = nth_RT - avg_SSD

    return SSRT

## 

In [4]:
df = pd.read_csv(data_files[0])

In [5]:
df

,corrGoResp,shape,practiceBlocks.thisRepN,practiceBlocks.thisTrialN,practiceBlocks.thisN,practiceBlocks.thisIndex,practiceTrials.thisRepN,practiceTrials.thisTrialN,practiceTrials.thisN,practiceTrials.thisIndex,...,end_resp.started,end_resp.keys,end_resp.rt,participant,session,date,expName,psychopyVersion,frameRate,Unnamed: 88
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,s004,1,2024-06-19_14h58.56.166,simplestop_skeleton,2022.2.2,60.206955,NaN
1,None,None,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,NaN,NaN,NaN,s004,1,2024-06-19_14h58.56.166,simplestop_skeleton,2022.2.2,60.206955,NaN
2,None,None,0.0,0.0,0.0,0.0,0.0,1.0,1.0,3.0,...,NaN,NaN,NaN,s004,1,2024-06-19_14h58.56.166,simplestop_skeleton,2022.2.2,60.206955,NaN
3,None,None,0.0,0.0,0.0,0.0,0.0,2.0,2.0,3.0,...,NaN,NaN,NaN,s004,1,2024-06-19_14h58.56.166,simplestop_skeleton,2022.2.2,60.206955,NaN
4,None,None,0.0,0.0,0.0,0.0,0.0,3.0,3.0,3.0,...,NaN,NaN,NaN,s004,1,2024-06-19_14h58.56.166,simplestop_skeleton,2022.2.2,60.206955,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,s004,1,2024-06-19_14h58.56.166,simplestop_skeleton,2022.2.2,60.206955,NaN
170,$currentCorrectResponse,$currentGoStim,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,s004,1,2024-06-19_14h58.56.166,simplestop_skeleton,2022.2.2,60.206955,NaN
171,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,s004,1,2024-06-19_14h58.56.166,simplestop_skeleton,2022.2.2,60.206955,NaN
172,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,s004,1,2024-06-19_14h58.56.166,simplestop_skeleton,2022.2.2,60.206955,NaN
